# MongoDB 01 - Shell 사용법

# 데이터 디렉토리 (사용자 권한)
* (관리자가 아닌) 사용자 권한으로만 MongoDB를 실행하고자 하는 경우
* 다음과 같이 데이터 디렉토리를 생성

```bash
# 사용자 디렉토리에 'db' 폴더를 생성
$ mkdir ~/db  
```

# MongoDB 데몬 시작과 종료

```bash
# 시작: 데몬을 실행하면서 데이터 저장 경로를 지정
$ mongod --dbpath ~/db --nojournal & 

# 종료: shutdown 옵션을 사용
$ mongod --dbpath ~/db -shutdown
```

# 간단한 읽기 쓰기
* 다음은 MongoDB shell 를 실행하여 간단한 읽기 쓰기를 테스트 해본 것이다.
* MongoDB shell의 기본 문법은 JavaScript 문법을 기반으로 하고 있다.

```javascript
$ mongo
MongoDB shell version: 2.4.9
connecting to: test
Thu Sep 25 14:58:01.018 [initandlisten] connection accepted from 127.0.0.1:40734 #1 (1 connection now open)
> // 현재 데이터베이스 확인
> db
test
> // 도큐먼트 준비하고 저장
> doc = { "Ticker": "AAPL", "Company":"Apple Inc."}
{ "Ticker" : "AAPL", "Company" : "Apple Inc." }
> db.test.save(doc)
> db.test.save({"Ticker": "GOOG", "Company":"Google Inc."})
> // 도큐먼트 목록
> db.test.find()
{ "_id" : ObjectId("54279898c8a18f846c9cdf1f"), "Ticker" : "AAPL", "Company" : "Apple Inc." }
{ "_id" : ObjectId("542798b9c8a18f846c9cdf20"), "Ticker" : "GOOG", "Company" : "Google Inc." }
> // 도큐먼트 삭제
> db.test.remove({ "Ticker": "AAPL", "Company":"Apple Inc."})
> // 도큐먼트 목록
> db.test.find()
{ "_id" : ObjectId("542798b9c8a18f846c9cdf20"), "Ticker" : "GOOG", "Company" : "Google Inc." }
> 
```

기본적으로 "test"라는 이름을 가진 컬랙션이 생성되어 있다. 여기서 도큐먼트란 JSON 도큐먼트를 말한다.

* save(): 도큐먼트를 저장한다. 
* find(): 컬랙션을 검색한다. (인자를 지정하지 않으면 컬랙션내 모든 도큐먼트를 보여준다)
* remvoe(): 도큐먼트를 삭제한다.



# 데이터베이스, 컬렉션, 도큐먼트
MongoDB내의 데이터베이스 목록이나 데이터베이스 내의 컬랙션 목록을 살펴보려면 다음과 같이 한다.  

```javascript
> // 데이터베이스 목록 ("show dbs" 라고 쓸수도 있다)
> show databases
> // 데이터베이스 내의 컬랙션 목록
> show collections
```

# use
db는 현재 사용중인 데이터베이스를 가르킨다. 사용하는 데이터베이스를 바꾸려면 use 명령을 쓴다.

```javascript
> // 기본 데이터베이스 지정
> use stocks 
switched to db stocks
> // 기본 데이터베이스 지정 확인
> db  
stocks
```

거의 사용할 일이 없지만, 데이터베이스 전체를 삭제하려면 db.dropDatabase() 명령을 쓴다.
처음 테스트 삼아 데이터를 넣었다가 전체를 삭제할 때 사용하기도 한다.

```javascript
> db  
stocks
> // 데이터 베이스 전체 삭제
> db.dropDatabase()
Sat Sep 27 16:59:12.040 [conn7] dropDatabase stocks starting
Sat Sep 27 16:59:12.041 [conn7] removeJournalFiles
Sat Sep 27 16:59:12.071 [conn7] dropDatabase stocks finished
{ "dropped" : "stocks", "ok" : 1 }
```

## insert(), save(), update()의 차이

 insert()와 save()는 둘 다 도큐먼트를 컬랙션에 저장하는 메서드이지만 다소 차이가 있다. 차이점을 살펴보기 위해 우선 간단한 데이터를  준비해보자.

```javascript
> db.dropDatabase()
> db
test
> db.test.save({ "Ticker": "AAPL"})
> db.test.save({"Ticker": "GOOG"})
> db.test.find()
{ "_id" : ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker" : "AAPL" }
{ "_id" : ObjectId("5427a572c8a18f846c9cdf25"), "Ticker" : "GOOG" }
```

동일한  "_id"를 지정하여 insert()해보고, save()도 해보자.

```javascript
> db.test.insert({"_id":ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker":"AAPL", "Company" : "Apple Inc."})
E11000 duplicate key error index: test.test.$_id_  dup key: { : ObjectId('5427a56dc8a18f846c9cdf24') }
> 
```

# insert()

insert()에서는 키(\_id)가 중복되었다고 에러를 낸다. <br/>
insert() 도큐먼트를 최초 삽입할때 사용하며, 키(\_id)가 중복되지 않아야 한다. <br/>
만일 키(\_id)를 지정하지 않으면 insert()로 저장하면 자동으로 고유한 키값이 생성된다.

```javascript
> db.test.save({"_id":ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker":"AAPL", "Company" : "Apple Inc."})
> db.test.find()
{ "_id" : ObjectId("5427a572c8a18f846c9cdf25"), "Ticker" : "GOOG" }
{ "_id" : ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker" : "AAPL", "Company" : "Apple Inc." }
> 

```

# save()
save()는 도큐먼트 전체를 다시 저장한다. <br/>
즉, 덮어쓰기를 한다. 새로운 속성("Sector")과 값("Consumer Goods")을 추가하여 save() 하면, 새로운 속성과 함께 도큐먼트 저장된다. 

```javascript
> db.test.save({"_id":ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker":"AAPL", "Industry":"Electronic Equipment"})
> db.test.find()
{ "_id" : ObjectId("5427a572c8a18f846c9cdf25"), "Ticker" : "GOOG" }
{ "_id" : ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker" : "AAPL", "Industry" : "Electronic Equipment" }
> 
```

위 예를 살펴보면 save()한 후에 이전 도큐먼트에 있었던 "Company":"Apple Inc." 속성은 삭제 된 것을 확인할 수 있다. <br/>
다시말해, save()는 도큐먼트를 새로운 도큐먼트를 덮어쓴다.

# update()
update()는 조건에 맞는 도큐먼트를 찾아 필드별로 수정하는 기능을 한다. 부분 업데이를 하므로 save()에 비해 훨씬 빠르게 동작한다.

```javascript
> db.test.update({"Ticker":"GOOG"}, {$set: {"Sector":"Technology"}})
> db.test.find()
{ "_id" : ObjectId("5427a56dc8a18f846c9cdf24"), "Ticker" : "AAPL", "Industry" : "Electronic Equipment" }
{ "Sector" : "Technology", "Ticker" : "GOOG", "_id" : ObjectId("5427a572c8a18f846c9cdf25") }
> 
```

# 요약

주로 다음 세가지 메서드를 이용하여 도큐먼트 조작을 한다.

* insert() - 새로운 도큐먼트를 추가한다. (동일한 id로 추가할 수 없다)
* save() - 지정한 id를 가진 도큐먼트를 전체를 덮어쓴다
* update() - 특정 필드만 업데이트 한다.

도큐먼트를 추가할 때 insert() 혹은 save()를 쓸 수 있으며, 이 두 메서드의 차이를 이해하고 써야 한다.
도큐먼트를 수정할 때는 update()를 사용하여 수정이 필요한 특정 필드만 업데이트하는 것이 효과적이다.